In [1]:
import pandas as pd
import numpy as np

# Load data from the CSV file
data = pd.read_csv('data(pretrained).csv')

#Extract relevant columns
rouge_scores = data['Rouge_Scores'].values
length_of_summary = data['Length_of_Summary'].values
training_time = data['Training_Time'].values

# Weights for each parameter
weights = np.array([0.5, 0.25, 0.25])

# Normalize the matrix
normalized_matrix = np.column_stack([
    rouge_scores / np.max(rouge_scores),
    1 - (length_of_summary / np.max(length_of_summary)),
    1 - (training_time / np.max(training_time))
])

# Calculate the weighted normalized decision matrix
weighted_normalized_matrix = normalized_matrix * weights

# Ideal and Negative Ideal solutions
ideal_solution = np.max(weighted_normalized_matrix, axis=0)
negative_ideal_solution = np.min(weighted_normalized_matrix, axis=0)

# Calculate the separation measures
distance_to_ideal = np.sqrt(np.sum((weighted_normalized_matrix - ideal_solution)**2, axis=1))
distance_to_negative_ideal = np.sqrt(np.sum((weighted_normalized_matrix - negative_ideal_solution)**2, axis=1))

# Calculate the TOPSIS scores
topsis_scores = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)

# Rank the models based on TOPSIS scores
data['TOPSIS_Score'] = topsis_scores
data['Rank'] = data['TOPSIS_Score'].rank(ascending=False)

# Print the results
print("Model Ranking:")
print(data[['Model', 'TOPSIS_Score', 'Rank']].sort_values(by='Rank'))



Model Ranking:
        Model  TOPSIS_Score  Rank
2  DistilBERT      0.682101   1.0
5        CTRL      0.677613   2.0
4       GPT-2      0.619290   3.0
0     RoBERTa      0.511416   4.0
1     ELECTRA      0.420252   5.0
3      ALBERT      0.317899   6.0


In [2]:
data.head()

,Model,Rouge_Scores,Length_of_Summary,Training_Time,TOPSIS_Score,Rank
0,RoBERTa,0.77,133,10,0.511416,4.0
1,ELECTRA,0.79,137,11,0.420252,5.0
2,DistilBERT,0.72,125,7,0.682101,1.0
3,ALBERT,0.81,142,13,0.317899,6.0
4,GPT-2,0.76,132,9,0.619290,3.0
